In [2311]:
import pandas as pd

In [2312]:
pd.set_option('display.max_columns', None)
store_records = pd.read_csv('../data/sample_superstore_updated.csv')
store_records['order_year'] = store_records['order_date'].str[-4:]
store_records.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,year,order_year
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,2016,2016
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,2016,2016
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,2016,2016
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,2015,2015
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,2015,2015


In [2313]:
store_records['order_date'] = pd.to_datetime(store_records['order_date'])

## KPI Data Points for Dashboard

In [2314]:
# sales, are they up? # sales are up 51% in the last 4 years
sales_delta = store_records[['order_id','sales','year']].groupby('year').agg({'sales':'sum','order_id':'first'}).reset_index()
sales_2017 = int(sales_delta['sales'].iloc[3])
sales_2016 = int(sales_delta['sales'].iloc[2])
sales_2014 = int(sales_delta['sales'].iloc[0])

sales_pct_change = (sales_2017 - sales_2014)/sales_2014 * 100
sales_pct_change

51.41343157520852

In [2315]:
# sales are up 20% in last year, from 2016 to 2017
sales_pct_change_recent = (sales_2017 - sales_2016)/sales_2016 * 100
sales_pct_change_recent

20.35603778695185

In [2316]:
# profits, are they up? # profits are up 88% in the last 4 years
profit_delta = store_records[['order_id','profit','year']].groupby('year').agg({'profit':'sum','order_id':'first'}).reset_index()
profits_2017 = int(profit_delta['profit'].iloc[3])
profits_2016 = int(profit_delta['profit'].iloc[2])
profits_2014 = int(profit_delta['profit'].iloc[0])

profit_pct_change = (profits_2017 - profits_2014)/profits_2014 * 100
profit_pct_change

88.60182064065559

In [2317]:
# profits are up 14% in last year, from 2016 to 2017
profit_pct_change_recent = (profits_2017 - profits_2016)/profits_2016 * 100
profit_pct_change_recent

14.235588972431076

In [2318]:
# year/year sales trend

sales_delta

,year,sales,order_id
0,2014,484247.4981,CA-2014-115812
1,2015,470532.5090,US-2015-108966
2,2016,609205.5980,CA-2016-152156
3,2017,733215.2552,CA-2017-114412


In [2319]:
# total sales
total_sales = int(store_records['sales'].sum())
total_items_sold = int(store_records['quantity'].sum())
print(f"Total sales: ${total_sales}.")
print(f"Total quantity: {total_items_sold}.")

Total sales: $2297200.
Total quantity: 37873.


In [2320]:
# category of most sold products (rank top 3)
# there are only 3 main categories, but this is where each ranks

top_3_category = store_records[['product_id', 'quantity', 'product_name','category','sub_category']].groupby('category').agg({'product_id':'count','quantity':'sum'}).rename(columns={'product_id':'product_id_count'}).sort_values(by='quantity', ascending=False).reset_index()
top_3_category

,category,product_id_count,quantity
0,Office Supplies,6026,22906
1,Furniture,2121,8028
2,Technology,1847,6939


In [2321]:
# category of most sold products (rank top 5)

top_5_sub_category = store_records[['product_id', 'quantity', 'product_name','category','sub_category']].groupby('sub_category').agg({'product_id':'count', 'quantity':'sum'}).rename(columns={'product_id':'product_id_count'}).sort_values(by='quantity', ascending=False).reset_index()
top_5_sub_category = top_5_sub_category.iloc[0:5]
top_5_sub_category

,sub_category,product_id_count,quantity
0,Binders,1523,5974
1,Paper,1370,5178
2,Furnishings,957,3563
3,Phones,889,3289
4,Storage,846,3158


In [2322]:
# top 5 list of products that generate the most profit

high_profit_products = store_records[['product_id', 'product_name','category','sub_category','profit']].groupby('product_id').agg({'profit':'sum', 'product_name':'first', 'category':'first', 'sub_category':'first'}).sort_values(by='profit', ascending=False).reset_index()
high_profit_products_top_5 = high_profit_products.iloc[0:5]
high_profit_products_top_5

,product_id,profit,product_name,category,sub_category
0,TEC-CO-10004722,25199.9280,Canon imageCLASS 2200 Advanced Copier,Technology,Copiers
1,OFF-BI-10003527,7753.0390,Fellowes PB500 Electric Punch Plastic Comb Bin...,Office Supplies,Binders
2,TEC-CO-10001449,6983.8836,Hewlett Packard LaserJet 3310 Copier,Technology,Copiers
3,TEC-CO-10003763,4570.9347,Canon PC1060 Personal Laser Copier,Technology,Copiers
4,TEC-AC-10002049,4425.3432,Logitech G19 Programmable Gaming Keyboard,Technology,Accessories


In [2323]:
# top 5 list of products that generate the least profit

neg_profit_products = store_records[['product_id', 'product_name','category','sub_category','profit']].groupby('product_id').agg({'profit':'sum', 'product_name':'first', 'category':'first', 'sub_category':'first'}).sort_values(by='profit', ascending=True).reset_index()
neg_profit_products_top_5 = neg_profit_products.iloc[0:5]
neg_profit_products_top_5

,product_id,profit,product_name,category,sub_category
0,TEC-MA-10000418,-8879.9704,Cubify CubeX 3D Printer Double Head Print,Technology,Machines
1,TEC-MA-10000822,-4589.9730,Lexmark MX611dhe Monochrome Laser Printer,Technology,Machines
2,TEC-MA-10004125,-3839.9904,Cubify CubeX 3D Printer Triple Head Print,Technology,Machines
3,FUR-TA-10000198,-2876.1156,Chromcraft Bull-Nose Wood Oval Conference Tabl...,Furniture,Tables
4,FUR-TA-10001889,-1934.3976,Bush Advantage Collection Racetrack Conference...,Furniture,Tables


In [2324]:
# states that place the most orders

high_business_states = store_records[['product_id','quantity','state','region']].groupby('state').agg({'product_id':'count'}).sort_values(by='product_id', ascending=False).reset_index()
high_business_states = high_business_states.iloc[0:5]
high_business_states

,state,product_id
0,California,2001
1,New York,1128
2,Texas,985
3,Pennsylvania,587
4,Washington,506


In [2325]:
# regions that place the most orders

high_business_states = store_records[['product_id','state','region']].groupby('region').agg({'product_id':'count'}).sort_values(by='product_id', ascending=False).reset_index()
high_business_states = high_business_states.iloc[0:4]
high_business_states

,region,product_id
0,West,3203
1,East,2848
2,Central,2323
3,South,1620


In [2326]:
# popular orders by region

popular_by_region = store_records[['product_id','quantity','category','sub_category','region']].groupby(['region','category']).agg({'product_id':'count', 'quantity':'sum'}).sort_values(by='product_id', ascending=False).reset_index()
popular_by_region = popular_by_region.rename(columns={'product_id':'product_id_count'})
popular_by_region = popular_by_region.iloc[0:5]
popular_by_region

,region,category,product_id_count,quantity
0,West,Office Supplies,1897,7235
1,East,Office Supplies,1712,6462
2,Central,Office Supplies,1422,5409
3,South,Office Supplies,995,3800
4,West,Furniture,707,2696


In [2327]:
# states that purchase the most products 

high_business_states_q = store_records[['product_id','quantity','state','region']].groupby('state').agg({'quantity':'sum'}).sort_values(by='quantity', ascending=False).reset_index()
high_business_states_q = high_business_states_q.iloc[0:5]
high_business_states_q

,state,quantity
0,California,7667
1,New York,4224
2,Texas,3724
3,Pennsylvania,2153
4,Washington,1883


In [2328]:
# regions that purchase the most products 

high_business_states = store_records[['product_id','quantity','state','region']].groupby('region').agg({'product_id':'count', 'quantity':'sum'}).sort_values(by='product_id', ascending=False).reset_index()
high_business_states = high_business_states.rename(columns={'product_id':'product_id_count'})
high_business_states = high_business_states.iloc[0:4]
high_business_states

,region,product_id_count,quantity
0,West,3203,12266
1,East,2848,10618
2,Central,2323,8780
3,South,1620,6209


In [2329]:
# top 8 overall popular products

popular_products = store_records[['product_id','quantity','state','product_name']].groupby(['product_id']).agg({'quantity':'sum', 'product_name':'first'}).sort_values(by='quantity', ascending=False).reset_index()
popular_products = popular_products.iloc[0:8]
popular_products

,product_id,quantity,product_name
0,TEC-AC-10003832,75,Logitech P710e Mobile Speakerphone
1,OFF-PA-10001970,70,Xerox 1881
2,OFF-BI-10001524,67,GBC Premium Transparent Covers with Diagonal L...
3,FUR-CH-10002647,64,"Situations Contoured Folding Chairs, 4/Set"
4,OFF-BI-10002026,64,Avery Arch Ring Binders
5,FUR-TA-10001095,61,Chromcraft Round Conference Tables
6,TEC-AC-10002049,60,Logitech G19 Programmable Gaming Keyboard
7,OFF-BI-10004728,59,Wilson Jones Turn Tabs Binder Tool for Ring Bi...


In [2330]:
# popular products by region

popular_products_region = store_records[['product_id','quantity','state','region','product_name']].groupby(['product_id','region']).agg({'quantity':'sum', 'product_name':'first'}).sort_values(by='quantity', ascending=False).reset_index()
popular_products_region

,product_id,region,quantity,product_name
0,TEC-AC-10003832,West,45,Logitech P710e Mobile Speakerphone
1,OFF-BI-10000301,Central,34,GBC Instant Report Kit
2,OFF-BI-10000756,Central,33,Storex DuraTech Recycled Plastic Frosted Binders
3,OFF-PA-10001970,East,33,Xerox 1881
4,OFF-BI-10003656,East,32,Fellowes PB200 Plastic Comb Binding Machine
...,...,...,...,...
5293,OFF-PA-10000048,West,1,Xerox 20
5294,OFF-BI-10000309,West,1,"GBC Twin Loop Wire Binding Elements, 9/16"" Spi..."
5295,OFF-BI-10000315,Central,1,Poly Designer Cover & Back
5296,OFF-BI-10004738,West,1,Flexible Leather- Look Classic Collection Ring...


In [2331]:
# popular products in the west
popular_products_region[popular_products_region['region'] == 'West'].head(5)

,product_id,region,quantity,product_name
0,TEC-AC-10003832,West,45,Logitech P710e Mobile Speakerphone
5,OFF-BI-10000174,West,32,Wilson Jones Clip & Carry Folder Binder Tool f...
8,OFF-BI-10001036,West,31,Cardinal EasyOpen D-Ring Binders
13,OFF-BI-10001670,West,29,Vinyl Sectional Post Binders
14,OFF-ST-10002486,West,29,Eldon Shelf Savers Cubes and Bins


In [2332]:
# profits by shipping type

high_profit_shipping = store_records[['product_id','ship_mode','profit']].groupby('ship_mode').agg({'profit':'sum', 'product_id':'count'}).sort_values(by='profit', ascending=False).reset_index()
high_profit_shipping = high_profit_shipping.rename(columns={'product_id':'order_count'})
high_profit_shipping

,ship_mode,profit,order_count
0,Standard Class,164088.7875,5968
1,Second Class,57446.6354,1945
2,First Class,48969.8399,1538
3,Same Day,15891.7589,543


In [2333]:
# high profit products  (rank top 5, sub-category)

top_5_sub_category_profit = store_records[['product_id', 'profit', 'product_name','category','sub_category']].groupby('sub_category').agg({'product_id':'count', 'profit':'sum'}).rename(columns={'product_id':'product_id_count'}).sort_values(by='profit', ascending=False).reset_index()
top_5_sub_category_profit = top_5_sub_category_profit.iloc[0:5]
top_5_sub_category_profit

,sub_category,product_id_count,profit
0,Copiers,68,55617.8249
1,Phones,889,44515.7306
2,Accessories,775,41936.6357
3,Paper,1370,34053.5693
4,Binders,1523,30221.7633


In [2334]:
# high profit products  (rank top 3, category)

top_3_category_profit = store_records[['product_id', 'profit', 'product_name','category']].groupby('category').agg({'product_id':'count', 'profit':'sum'}).rename(columns={'product_id':'product_id_count'}).sort_values(by='profit', ascending=False).reset_index()
top_3_category_profit = top_3_category_profit.iloc[0:5]
top_3_category_profit

,category,product_id_count,profit
0,Technology,1847,145454.9481
1,Office Supplies,6026,122490.8008
2,Furniture,2121,18451.2728


In [2335]:
# top 10 products by overall profit

top_10_sub_category_profit = store_records[['product_id', 'profit', 'product_name','category']].groupby('product_id').agg({'product_id':'count', 'profit':'sum', 'product_name':'first', 'category':'first'}).rename(columns={'product_id':'product_id_count'}).sort_values(by='profit', ascending=False).reset_index()
top_10_sub_category_profit = top_10_sub_category_profit.iloc[0:10]
top_10_sub_category_profit

,product_id,product_id_count,profit,product_name,category
0,TEC-CO-10004722,5,25199.9280,Canon imageCLASS 2200 Advanced Copier,Technology
1,OFF-BI-10003527,10,7753.0390,Fellowes PB500 Electric Punch Plastic Comb Bin...,Office Supplies
2,TEC-CO-10001449,8,6983.8836,Hewlett Packard LaserJet 3310 Copier,Technology
3,TEC-CO-10003763,4,4570.9347,Canon PC1060 Personal Laser Copier,Technology
4,TEC-AC-10002049,15,4425.3432,Logitech G19 Programmable Gaming Keyboard,Technology
5,TEC-MA-10001127,3,4094.9766,HP Designjet T520 Inkjet Large Format Printer ...,Technology
6,TEC-MA-10003979,2,3772.9461,Ativa V4110MDD Micro-Cut Shredder,Technology
7,TEC-MA-10001047,2,3717.9714,"3D Systems Cube Printer, 2nd Generation, Magenta",Technology
8,OFF-BI-10001120,3,3345.2823,Ibico EPK-21 Electric Binding System,Office Supplies
9,TEC-MA-10000045,2,3343.5360,Zebra ZM400 Thermal Label Printer,Technology


In [2336]:
# top 10 products by profit per unitf

top_10_sub_category_profit2 = store_records[['product_id', 'profit', 'product_name','category','quantity']].groupby('product_id').agg({'product_id':'count', 'profit':'sum', 'product_name':'first', 'category':'first', 'quantity':'sum'}).rename(columns={'product_id':'product_id_count'}).sort_values(by='profit', ascending=False).reset_index()
top_10_sub_category_profit2 = top_10_sub_category_profit2.iloc[0:10]
top_10_sub_category_profit2['per_unit_profit'] = (top_10_sub_category_profit2['profit'] / top_10_sub_category_profit2['quantity'])
top_10_sub_category_profit2['per_unit_profit'] = top_10_sub_category_profit2['per_unit_profit'].round(2)
top_10_sub_category_profit2 = top_10_sub_category_profit2.drop(columns=['product_id_count','product_id'])
top_10_sub_category_profit2.sort_values(by='per_unit_profit', ascending=False)

,profit,product_name,category,quantity,per_unit_profit
0,25199.9280,Canon imageCLASS 2200 Advanced Copier,Technology,20,1260.00
9,3343.5360,Zebra ZM400 Thermal Label Printer,Technology,6,557.26
6,3772.9461,Ativa V4110MDD Micro-Cut Shredder,Technology,11,343.00
5,4094.9766,HP Designjet T520 Inkjet Large Format Printer ...,Technology,12,341.25
7,3717.9714,"3D Systems Cube Printer, 2nd Generation, Magenta",Technology,11,338.00
8,3345.2823,Ibico EPK-21 Electric Binding System,Office Supplies,13,257.33
1,7753.0390,Fellowes PB500 Electric Punch Plastic Comb Bin...,Office Supplies,31,250.10
3,4570.9347,Canon PC1060 Personal Laser Copier,Technology,19,240.58
2,6983.8836,Hewlett Packard LaserJet 3310 Copier,Technology,38,183.79
4,4425.3432,Logitech G19 Programmable Gaming Keyboard,Technology,60,73.76


In [2337]:
# gross rate of revenue per item or gross rate for sales quantity

gross_rate_per_unit = int(store_records['sales'].sum() / store_records['quantity'].sum())
gross_rate_per_unit

60

In [2338]:
store_records['customer_id'].nunique()

793

In [2339]:
# retention rate - ie, how many of those customers have come back
# we have about 300 repeat customers that purchase every year out of 793 unique customers

retention_rate = store_records[['customer_id','year','product_id']].groupby(['year','customer_id']).agg('count').reset_index()
retention_rate_pv = pd.pivot(retention_rate, index='customer_id', columns='year', values='product_id').reset_index()
retention_rate_pv2 = retention_rate_pv.dropna(axis=0)
retention_rate_pv2

year,customer_id,2014,2015,2016,2017
0,AA-10315,4.0,1.0,4.0,2.0
1,AA-10375,2.0,4.0,4.0,5.0
3,AA-10645,6.0,3.0,8.0,1.0
6,AB-10105,2.0,6.0,3.0,9.0
7,AB-10150,5.0,1.0,1.0,5.0
...,...,...,...,...,...
786,VW-21775,3.0,7.0,3.0,5.0
787,WB-21850,8.0,6.0,16.0,7.0
788,XP-21865,10.0,3.0,9.0,6.0
789,YC-21895,3.0,1.0,3.0,1.0


In [2340]:
# and 772 repeat customers who have purchased from the store more than once

retention_rate_pv = retention_rate_pv.fillna(0)
retention_rate_pv['single_purchase_customer'] = retention_rate_pv[retention_rate_pv == 0].count(axis=1).gt(2)
repeat_customers = retention_rate_pv[retention_rate_pv['single_purchase_customer'] == False]

In [2341]:
len(repeat_customers)

772

In [2342]:
repeat_customers_ls = list(repeat_customers['customer_id'].unique())

In [2343]:
# what % of purchases are repeated (customerID)
## make sure I didn't count unique orders without grouping
df_repeat_customers = store_records[store_records['customer_id'].isin(repeat_customers_ls)]
df_repeat_customers = df_repeat_customers[['order_id','customer_id', 'product_id']]
df_repeat_customers = df_repeat_customers.groupby(['customer_id', 'order_id']).agg('count').reset_index()

# after grouping, we have to subtract number of unique customers from unique orders
# unique orders is counting all orders
# for the purposes of calculating % of repeat orders, we can't count the first order
# hence, deleting one order for each customer or the number of unique customers removed from total unique orders
number_of_repeat_orders = len(df_repeat_customers['order_id'].unique()) - len(df_repeat_customers['customer_id'].unique())
number_of_unique_customers = len(df_repeat_customers['customer_id'].unique())
pct_repeated_orders = (number_of_repeat_orders - number_of_unique_customers) / store_records['order_id'].nunique()
pct_repeated_orders = round(pct_repeated_orders, 2) * 100
pct_repeated_orders

68.0

In [2344]:
# repeat customer orders & the products they buy
# not an analysis of the quantity of products, but the number of orders of each product

df_repeat_customer_products = store_records[store_records['customer_id'].isin(repeat_customers_ls)]
df_repeat_customer_products = df_repeat_customer_products[['order_id','customer_id', 'product_id','product_name']]
df_repeat_customer_products = df_repeat_customer_products.groupby(['product_id']).agg({'product_name':'first', 'order_id':'count'}).reset_index()
df_repeat_customer_products = df_repeat_customer_products.rename(columns={'order_id':'order_count'})
df_repeat_customer_products = df_repeat_customer_products.sort_values(by='order_count', ascending=False)
df_repeat_customer_products.iloc[0:10]

,product_id,product_name,order_count
1143,OFF-PA-10001970,Xerox 1881,19
295,FUR-FU-10004270,"Eldon Image Series Desk Accessories, Burgundy",16
1568,TEC-AC-10003832,Logitech P710e Mobile Speakerphone,16
65,FUR-CH-10001146,"Global Value Mid-Back Manager's Chair, Gray",15
1565,TEC-AC-10003628,Logitech 910-002974 M325 Wireless Mouse for We...,15
1516,TEC-AC-10002049,Logitech G19 Programmable Gaming Keyboard,15
93,FUR-CH-10002647,"Situations Contoured Folding Chairs, 4/Set",15
110,FUR-CH-10003774,"Global Wood Trimmed Manager's Task Chair, Khaki",14
721,OFF-BI-10002026,Avery Arch Ring Binders,14
183,FUR-FU-10001473,DAX Wood Document Frame,14


In [2345]:
df_repeat_customers

,customer_id,order_id,product_id
0,AA-10315,CA-2014-128055,2
1,AA-10315,CA-2014-138100,2
2,AA-10315,CA-2015-121391,1
3,AA-10315,CA-2016-103982,4
4,AA-10315,CA-2017-147039,2
...,...,...,...
4968,ZD-21925,CA-2014-143336,3
4969,ZD-21925,CA-2016-152471,2
4970,ZD-21925,CA-2016-167682,2
4971,ZD-21925,CA-2017-141481,1


In [2346]:
# list of products consistently in high demand across all years, started

in_demand_yearly = store_records[['product_id','quantity','category','sub_category','order_year']].groupby(['product_id','order_year']).agg({'quantity':'sum'}).reset_index()
in_demand_yearly = pd.pivot(in_demand_yearly, index='product_id', columns='order_year', values='quantity').reset_index()
in_demand_yearly = in_demand_yearly.fillna(0)
in_demand_yearly = in_demand_yearly.sort_values(by='2017', ascending=False)
in_demand_yearly

order_year,product_id,2014,2015,2016,2017
1507,TEC-AC-10001772,7.0,0.0,4.0,34.0
137,FUR-FU-10000023,3.0,3.0,8.0,31.0
327,FUR-TA-10001520,2.0,7.0,0.0,30.0
1168,OFF-PA-10002377,2.0,12.0,8.0,30.0
1583,TEC-AC-10004510,6.0,7.0,8.0,29.0
...,...,...,...,...,...
1625,TEC-MA-10000864,0.0,16.0,0.0,0.0
739,OFF-BI-10002437,0.0,4.0,5.0,0.0
1623,TEC-MA-10000752,0.0,0.0,8.0,0.0
1622,TEC-MA-10000597,0.0,5.0,0.0,0.0


In [2347]:
in_demand_yearly.sort_values(by='2016', ascending=False)

/Users/merceded1205/Library/Caches/pypoetry/virtualenvs/superstore-analysis-SzCM6WCM-py3.12/lib/python3.12/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


order_year,product_id,2014,2015,2016,2017
1569,TEC-AC-10003832,6.0,3.0,41.0,25.0
325,FUR-TA-10001095,0.0,2.0,39.0,20.0
311,FUR-FU-10004864,3.0,0.0,31.0,12.0
835,OFF-BI-10004728,5.0,12.0,30.0,12.0
55,FUR-CH-10000454,13.0,0.0,30.0,8.0
...,...,...,...,...,...
847,OFF-EN-10000461,2.0,14.0,0.0,11.0
1167,OFF-PA-10002365,8.0,4.0,0.0,3.0
1166,OFF-PA-10002333,0.0,0.0,0.0,3.0
1058,OFF-PA-10000556,0.0,6.0,0.0,3.0
